In [9]:
import torch
import functools
import numpy  as np
from IPython.core.debugger import set_trace

Функция регистр добавляет в класс тензор метод mul, т.к.
мул наследуется от класса Function то он имеет метод apply
именно туда мы и перейдем

        - setattr(Tensor, name, functools.partialmethod(fn.apply, fn))
        - fn это мул <class '__main__.Mul'>
        - name = 'mul'
        - затем попав в метод apply (fn.apply)
            - arg(fn) это аргумент который мы передали в apply

и получается что  ctx это класс MUl, вызывает класс Tensor и передаем в него 
        
        - Tensor(arg.forward(ctx, self.data, *[t.data for t in x]))
        - self this a = 1
        - arg <class '__main__.Mul'>
        - x = 3 or b
        - self.data => 1.data=> self this tensor after .data get array [1]
        - 'подставим'
        - Mul.forward (x= ( self.data, y = *[t.data for t in x]))
            - save x, y
            - mul x * y 
- ctx это класс мул он передается для сохранения значений x, y
(method save tensor)
- затем мы сохраняем ctx  в переменную _ctx  класса тензор
- затем если мы выполним метод backward in class Tensor
произойдет следующие

        - пока не вызвали(backward) в классе Tensor есть
        только результат мул и сохраненный класс мул
        - и так вызываем

        if self._ctx is None:
            return
            проходим тк не нон
        if self.grad is None and fill:
            # iniy first grad ones
            assert self.data.size == 1
            self.grad = np.ones_like(self.data)      
        assert(self.grad is not None) 
        
        инициализируем градиент единицами, self.grad = [1]

        grads = self._ctx.backward(self._ctx, self.grad)
        
        grads = (array([3]), array([1]))
        - Mul.backward
        
            def backward(ctx, grad_out):      
                x, y = ctx.saved_tensors =>  [array([1]), array([3])] 
                return y * grad_out, x * grad_out

        получаем сохраненные значения изначальные
        затем умножаем на градиент он равен 1 пока что
        и возвращаем полученные значения
        ctx.parents  это тензоры а и б
            
            if len(self._ctx.parents) == 1:
                grads = [grads]

        это сделано чтоб использовать зип
            
            - self._ctx.parents (tensor([1]), tensor([3])),
            - grads = (array([3]), array([1]))

            for t, g in zip(self._ctx.parents, grads):
                # t  Tensor
                if g.shape != t.data.shape:
                    print('grad shape must match tensor shape')
                    assert(False)           
                t.grad = g

In [10]:
# make base class
class Tensor:
    def __init__(self, data):
        if type(data) != np.ndarray:
            print('error type data %r' % data)
            assert(False)
        self.data = data        
        self.grad = None
        self._ctx = None
    
    def __repr__(self):
        # tensor([1.]))
        return f'tensor({self.data})'

    def backward(self, fill = True):
        
        if self._ctx is None:
            return

        if self.grad is None and fill:
            # iniy first grad ones
            assert self.data.size == 1
            self.grad = np.ones_like(self.data)
        
        assert(self.grad is not None)  

        """
        _ctx.backward return from <class '__main__.Mul'> 
        and method Mul.backward return y * grad_out, x * grad_out
        
        """      
        grads = self._ctx.backward(self._ctx, self.grad)
        if len(self._ctx.parents) == 1:
            grads = [grads]
       
        for t, g in zip(self._ctx.parents, grads):
            # t  Tensor
            if g.shape != t.data.shape:
                print('grad shape must match tensor shape')
                assert(False)
            t.grad = g          
      
         

"""
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html#:~:text=beginner%2Fexamples_autograd%2Ftwo_layer_net_custom_function-,PyTorch%3A%20Defining%20New%20autograd%20Functions,PyTorch%20autograd%20to%20compute%20gradients.
В прямом проходе мы получаем тензор, содержащий ввод и возврат
Тензор, содержащий вывод. 

ctx - это объект контекста, который можно использовать
хранить информацию для обратных вычислений. Вы можете кешировать произвольные
объекты для использования в обратном проходе с помощью метода ctx.save_for_backward.


При обратном проходе мы получаем тензор, содержащий градиент потери
относительно выхода, и нам нужно вычислить градиент потерь относительно входа.

"""

class Function:

    def __init__(self, *tensor):
        self.parents = tensor
        self.saved_tensors = []
    
    def save_for_backward(self, *x):
        self.saved_tensors.extend(x)    

    def apply(self, arg, *x):
        """
        почему мы здесь а потому что fn.apply и передаем аргумент fn
        a = Tensor(np.array([1]))
        b = Tensor(np.array([3]))
        
        a.mul(b)
        
        arg: <class '__main__.Mul'> , fn
        self.data -> Tensor.data->[1] a
        [3] = b        
        
        """        
        ctx = arg(self, *x)         
        ret = Tensor(arg.forward(ctx, self.data, *[t.data for t in x]))
        ret._ctx = ctx
        return ret


def register(name, fn):
    """
    class A:
        print('hell')
    a = A()
    setattr(a, 'oo', lambda x: x *2)
    a.oo(2)
    >4

    we add mul to class Tensor    

    partialmethod(fumc, arg)
    """    
    # set_trace()
    setattr(Tensor, name, functools.partialmethod(fn.apply, fn))


class Mul(Function):
    """
    out = x.mul.y
    back
    out/dx, out/dy
    
    """

    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x * y

    @staticmethod
    def backward(ctx, grad_out):       
        # set_trace()
        x, y = ctx.saved_tensors       
        return y * grad_out, x * grad_out

register('mul', Mul)

In [24]:
a = Tensor(np.array([1]))
b = Tensor(np.array([3]))
c = a.mul(b)
# check how works
print('No grad: ', a.grad, b.grad)
c.backward()
print('Numpy grad: ', c)
print('Numpy Grad: ', a.grad, b.grad)
print('-----------------')
print('Time to torch')
a  = torch.tensor([1.], requires_grad=True)
b  = torch.tensor([3.], requires_grad=True)
c = a.matmul(b)
c.backward()
print('Torch grad: ', c)
print('Torch Grad: ', a.grad, b.grad)

No grad:  None None
numpy grad:  tensor([3])
Grad:  [3] [1]
-----------------
Time to torch
Torch grad:  tensor(3., grad_fn=<DotBackward>)
Torch Grad:  tensor([3.]) tensor([1.])


In [ ]:
class Add(Function):
    """sum"""
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x + y

    @staticmethod
    def backward(ctx, grad_out):     
        x, y = ctx.saved_tensors       
        return grad_out, grad_out


class ReLU(Function):
    """relu"""
    @staticmethod
    def forward(ctx, in_val):
        ctx.save_for_backward(in_val)
        return np.maximum(in_val, 0)

    @staticmethod
    def backward(ctx, grad_out):     
        in_val = ctx.saved_tensors   
        grad_out[in_val < 0] = 0   
        return grad_out
        

class Sum(Function):
    """sum array each dx is 1"""
    @staticmethod
    def forward(ctx, in_val):
        ctx.save_for_backward(in_val)
        return np.sum(in_val)

    @staticmethod
    def backward(ctx, grad_out):     
        in_val = ctx.saved_tensors       
        return grad_out * np.ones_like(in_val)

class Dot(Function):
    """
    a = [[1, 0], [0, 1]]
    b = [[4, 1], [2, 2]]
    np.dot(a, b)
    >> array([[4, 1],
             [2, 2]])  
    
    """
    @staticmethod
    def forward(ctx, in_val, w):
        ctx.save_for_backward(in_val, w)
        return in_val.dot(w)

    @staticmethod
    def backward(ctx, grad_out):     
        in_val, w = ctx.saved_tensors       
        g_grad = grad_out.T.dot(in_val)
        w_grad = grad_out.T.dot(w)
        return g_grad, w_grad

register('dot', Dot)

TODO:

release train with tensor

        from mnist.ipynb

        np.random.seed(13)
        lr = 0.001
        batch = 30
        #init weight
        l1 = weight_init(784, 128, mode = 'uniform')
        l2 = weight_init(128, 10, mode = 'uniform')

        bar = trange(500)
        accuracies, losses = [], []
        for i in bar:
            samp = np.random.randint(0, x.shape[0], size=(batch))
            X = x[samp].reshape(-1, 28*28).float()
            Y = y[samp].long()

            x_loss, out, d_l1, d_l2 = backward(X.numpy(), Y.numpy(), l1, l2)

            cat = np.argmax(out, axis = 1)
            acc = (cat == Y.numpy()).mean()

            # update weight,  SGD

            l1 = l1 - lr * d_l1
            l2 = l2 - lr * d_l2

            x_loss = x_loss.mean()
            accuracies.append(acc)
            losses.append(x_loss)
            bar.set_description('Loss:  %.3f, Accuracy: %.3f' % (x_loss, acc))
        figsize(5, 5)
        plot(losses)
        plot(accuracies)


holy crap make logsoftmax

In [ ]:
def log_softmax_npy(x):
    # np.exp is not stable because it has Inf. So you should subtract maximum in x.
    # http://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    # x numpy   
    mx = x.max(axis=1)
    stbl = mx + np.log(np.exp(x- mx.reshape((-1, 1))).sum(axis=1))
    return x - stbl.reshape(-1,1)

In [ ]:
class Log_softmax(Function):

    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        mx = x.max(axis=1)
        stbl = mx + np.log(np.exp(x- mx.reshape((-1, 1))).sum(axis=1))
        return x - stbl.reshape(-1,1)
        

    @staticmethod
    def backward(ctx, grad_out):     
        x = ctx.saved_tensors 
        return  grad_out - np.exp(x)*grad_out.sum(axis=1).reshape((-1, 1))      
 

register('log_softmax', Log_softmax)